In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_model_snr', 'koi_fpflag_co','koi_fpflag_ss', 'koi_prad']]

In [5]:
selected_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   koi_fpflag_nt  6991 non-null   int64  
 1   koi_model_snr  6991 non-null   float64
 2   koi_fpflag_co  6991 non-null   int64  
 3   koi_fpflag_ss  6991 non-null   int64  
 4   koi_prad       6991 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 327.7 KB


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
target=df['koi_disposition']
target.head()

0         CONFIRMED
1    FALSE POSITIVE
2    FALSE POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: koi_disposition, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(selected_features,target,random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_model_snr,koi_fpflag_co,koi_fpflag_ss,koi_prad
6122,0,10.8,0,0,1.24
6370,0,13.8,0,1,0.86
2879,1,254.3,0,0,3.21
107,0,38.4,0,0,2.25
29,0,696.5,0,0,12.21


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler=MinMaxScaler().fit(X_train)
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)

# Train the Model



In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf = rf.fit(X_train_scaled, y_train)

In [11]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 0.9858859431623117
Testing Data Score: 0.8569794050343249


In [12]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.1871942 , 0.28282246, 0.17963033, 0.1328913 , 0.21746171])

In [13]:
# Sort the features by their importance
sorted(zip(rf.feature_importances_, selected_features), reverse=True)

[(0.282822464006703, 'koi_model_snr'),
 (0.217461705478109, 'koi_prad'),
 (0.18719419947041444, 'koi_fpflag_nt'),
 (0.17963033356738595, 'koi_fpflag_co'),
 (0.13289129747738762, 'koi_fpflag_ss')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [200, 300, 600],
              'max_features':['auto','sqrt','log2'],
              'max_depth': [4, 5, 6,7,8],
             'criterion':['gini','entropy']}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [15]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.890, total=   0.5s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  criterion=gini, max_depth=4, max_features=auto, n_estimators=200, score=0.880, total=   0.5s
[CV] criterion=gini, max_depth=4, max_features=auto, n_estimators=200 


...

[CV]  criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600, score=0.888, total=   1.9s
[CV] criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600, score=0.876, total=   1.9s
[CV] criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600, score=0.861, total=   2.1s
[CV] criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600, score=0.865, total=   2.0s
[CV] criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=600, score=0.861, total=   1.9s
[CV] criterion=entropy, max_depth=6, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=6, max_features=log2, n_estimators=200, score=0.888, total=   0.7s
[CV] criterion=ent

[CV]  criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=600, score=0.864, total=   2.0s
[CV] criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=600, score=0.865, total=   1.9s
[CV] criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=600 
[CV]  criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=600, score=0.863, total=   2.1s
[CV] criterion=entropy, max_depth=7, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=7, max_features=log2, n_estimators=200, score=0.890, total=   0.8s
[CV] criterion=entropy, max_depth=7, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=7, max_features=log2, n_estimators=200, score=0.880, total=   0.7s
[CV] criterion=entropy, max_depth=7, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=7, max_features=log2, n_estimators=200, score=0.867, total=   0.7s
[CV] criterion=ent

[CV]  criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=600, score=0.861, total=   2.2s
[CV] criterion=entropy, max_depth=8, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=8, max_features=log2, n_estimators=200, score=0.890, total=   0.7s
[CV] criterion=entropy, max_depth=8, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=8, max_features=log2, n_estimators=200, score=0.885, total=   0.7s
[CV] criterion=entropy, max_depth=8, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=8, max_features=log2, n_estimators=200, score=0.869, total=   0.7s
[CV] criterion=entropy, max_depth=8, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=8, max_features=log2, n_estimators=200, score=0.862, total=   0.7s
[CV] criterion=entropy, max_depth=8, max_features=log2, n_estimators=200 
[CV]  criterion=entropy, max_depth=8, max_features=log2, n_estimators=200, score=0.860, total=   0.7s
[CV] criterion=ent

[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:  8.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [16]:
# List the best parameters for this dataset
print(f"Best Parameters: {grid.best_params_}")

# List the best score
print(f"Best Score: {grid.best_score_}")

Best Parameters: {'criterion': 'gini', 'max_depth': 8, 'max_features': 'log2', 'n_estimators': 600}
Best Score: 0.8748753811336133


In [17]:
print(f"Training Grid Score: {grid.score(X_train_scaled, y_train)}")
print(f"Testing Grid Score: {grid.score(X_test_scaled, y_test)}")

Training Grid Score: 0.8806027083730689
Testing Grid Score: 0.879862700228833


In [18]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [19]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.81      0.69      0.75       411
     CONFIRMED       0.77      0.88      0.82       484
FALSE POSITIVE       0.98      0.97      0.97       853

      accuracy                           0.88      1748
     macro avg       0.85      0.85      0.85      1748
  weighted avg       0.88      0.88      0.88      1748



# Save the Model

In [20]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'adrianaovalle_rf_optimized.sav'
joblib.dump(rf, filename)

['adrianaovalle_rf_optimized.sav']